# How to use regioinvent

Note that running this entire notebook will take you around 2 hours.

To be able to use regioinvent, we will need:
- to install the brightway2 Python library (brightway2 and NOT brightway2.5), easier is to get it through activity-browser: https://github.com/LCA-ActivityBrowser/activity-browser
- a brightway project within which there is an ecoinvent database with either the version 3.9.1/3.10 cutoff
- to download the trade database stored here: https://doi.org/10.5281/zenodo.13146833 Make sure to take the "treated" database where UN COMTRADE data was refined

In [ ]:
import pandas as pd
import numpy as np
import sys
# change the path here to wherever you stored the Regioinvent Python package
sys.path.append(r'C:\Users\matth\PycharmProjects\Regiopremise')
import regioinvent

The initialization of the Regioinvent class requires 4 arguments:
- the name of the brightway2 project where you store ecoinvent and where regioinvent will be added
- the name you gave your ecoinvent database within your brightway2 project
- the version of the ecoinvent database ('3.9.1' or '3.10')

In [ ]:
ei_version = '3.10'

In [ ]:
regio = regioinvent.Regioinvent(
    bw_project_name=f'ecoinvent{ei_version}', 
    ecoinvent_database_name='ecoinvent_cutoff_3.10_image_SSP2-Base_2050+truck_carculator', 
    ecoinvent_version=ei_version,
)

First of regioinvent is to spatialize the original ecoinvent database. This entails multiple steps:
- Creating a new biosphere database for spatialized elementary flows (e.g., Ammonia, CA-QC)
- Spatializing the elementary flows used within the ecoinvent database, based on the location of the process itself
- Fixing water processes of ecoinvent creating imbalances (e.g., apple production in Chile calls irrigation, RoW -> imbalance)
- Importing LCIA methods supporting regionalization. Currently available: "IW v2.1". Soon available: "EF v3.1" and "ReCiPe2016 v1.1 (E)".

In [ ]:
regio.spatialize_my_ecoinvent()

This creates two databases in your brightway2 project:
- "biosphere3_spatialized_flows", which contains all the newly created spatialized elementary flows
- "the-name-of-your-ecoinvent-database regionalized", which is a copy of the original ecoinvent but using spatialized elementary flows and boasting new processes to fix water modeling

If you want to go further in the regionalization, i.e., to create new national production processes and to rely on trade data to regionalize consumption markets of ecoinvent, you can run the next function:

In [ ]:
regio.regionalize_ecoinvent_with_trade()

For more information on what these steps include, refer to the scientific article on regioinvent: _Agez, Maxime (2025) Regioinvent a regionalized version of ecoinvent integrating real trade data, International Journal of Life Cycle Assessment_

You can now go on brightway2 or AB to perform your LCAs normally with regioinvent.